## Customer Segmentation with RFM Analysis

### 1. Data Understanding

In [3]:
import pandas as pd 
import  datetime as dt

In [4]:
df_ = pd.read_excel("online_retail_II.xlsx")
df = df_.copy()

In [5]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [6]:
df["Description"].nunique()

4681

In [7]:
df["Description"].value_counts()

Description
WHITE HANGING HEART T-LIGHT HOLDER     3549
REGENCY CAKESTAND 3 TIER               2212
STRAWBERRY CERAMIC TRINKET BOX         1843
PACK OF 72 RETRO SPOT CAKE CASES       1466
ASSORTED COLOUR BIRD ORNAMENT          1457
                                       ... 
stock credited from royal yacht inc       1
VINTAGE METAL CAKE STAND CREAM            1
BLUE BAROQUE FLOCK CANDLE HOLDER          1
S/4 HEART CRYSTAL FRIDGE MAGNETS          1
dotcom email                              1
Name: count, Length: 4681, dtype: int64

In [8]:
df.groupby("Description")["Quantity"].sum().sort_values(ascending=False)

Description
WHITE HANGING HEART T-LIGHT HOLDER    57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS     54698
BROCADE RING PURSE                    47647
PACK OF 72 RETRO SPOT CAKE CASES      46106
ASSORTED COLOUR BIRD ORNAMENT         44925
                                      ...  
Zebra invcing error                   -9000
ebay sales                           -13630
missing                              -15149
given away                           -20000
?                                    -25373
Name: Quantity, Length: 4681, dtype: int64

In [9]:
df["Invoice"].nunique()

28816

In [10]:
df["TotalPrice"] = df["Price"] * df["Quantity"]

In [11]:
df.groupby("Invoice")["TotalPrice"].sum()

Invoice
489434      505.30
489435      145.80
489436      630.33
489437      310.75
489438     2286.24
            ...   
C538121     -12.75
C538122      -1.25
C538123      -7.50
C538124     -17.70
C538164      -1.95
Name: TotalPrice, Length: 28816, dtype: float64

### 2. Data Preparation

In [12]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
TotalPrice          0
dtype: int64

In [13]:
df.dropna(inplace = True)

In [14]:
df.describe(include = ["int","float"]).T

,count,mean,std,min,25%,50%,75%,max
Quantity,417534.0,12.758815,101.220424,-9360.00,2.00,4.00,12.00,19152.00
Price,417534.0,3.887547,71.131797,0.00,1.25,1.95,3.75,25111.09
Customer ID,417534.0,15360.645478,1680.811316,12346.00,13983.00,15311.00,16799.00,18287.00
TotalPrice,417534.0,19.994081,99.915863,-25111.09,4.25,11.25,19.35,15818.40


In the summary statistics table above, we observe negative minimum values for both the Quantity and TotalPrice variables, and there are also anomalies in the quartiles. This is because the dataset contains return (refund) invoices. Therefore, we will remove the return invoices from the dataset.

In [15]:
df = df[~df["Invoice"].astype("str").str.contains("C",na=False)]

### 3. Calculating RFM Metrics (Recency,Frequency,Monetary)

In [16]:
df["InvoiceDate"].max()

Timestamp('2010-12-09 20:01:00')

Since the last purchase date is December 9, 2010, we will set our reference date to two days later.

In [17]:
today_date = dt.datetime(2010,12,11)

In [18]:
rfm = df.groupby("Customer ID").agg({"InvoiceDate" :lambda date: (today_date-date.max()).days,
                               "Invoice" : lambda Invoice: Invoice.nunique(),
                               "TotalPrice":"sum"})

rfm.columns = ["recency","frequency","monetary"]
rfm.head()

,recency,frequency,monetary
Customer ID,,,
12346.0,165,11,372.86
12347.0,3,2,1323.32
12348.0,74,1,222.16
12349.0,43,3,2671.14
12351.0,11,1,300.93


In [19]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4314.0,91.269819,96.944304,1.0,18.00,53.00,136.0000,374.00
frequency,4314.0,4.454103,8.168658,1.0,1.00,2.00,5.0000,205.00
monetary,4314.0,2047.288659,8912.523243,0.0,307.95,705.55,1722.8025,349164.35


There are observations with a Monetary value of 0, and we will remove these from the dataset.

In [20]:
rfm = rfm[rfm["monetary"]>0]

### 4. Calculating RFM Scores

In [21]:
rfm["recency_score"] = pd.qcut(rfm["recency"],q=5,labels=[5,4,3,2,1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"),q=5,labels=[1,2,3,4,5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"],q=5,labels=[1,2,3,4,5]).astype(float).map("{:.2f}".format)
rfm["RFM_Score"] = rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str)
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_Score
Customer ID,,,,,,,
12346.0,165,11,372.86,2,5,2.00,25
12347.0,3,2,1323.32,5,2,4.00,52
12348.0,74,1,222.16,2,1,1.00,21
12349.0,43,3,2671.14,3,3,5.00,33
12351.0,11,1,300.93,5,1,2.00,51


### 5. Creating & Analysing RFM Segments

In [22]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm["segment"] = rfm["RFM_Score"].replace(seg_map,regex=True)
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_Score,segment
Customer ID,,,,,,,,
12346.0,165,11,372.86,2,5,2.00,25,cant_loose
12347.0,3,2,1323.32,5,2,4.00,52,potential_loyalists
12348.0,74,1,222.16,2,1,1.00,21,hibernating
12349.0,43,3,2671.14,3,3,5.00,33,need_attention
12351.0,11,1,300.93,5,1,2.00,51,new_customers


In [23]:
rfm.groupby("segment")[["recency","frequency","monetary"]].agg(["mean","count"])

recency        frequency           monetary      
                           mean count       mean count         mean count
segment                                                                  
about_to_sleep        53.819242   343   1.201166   343   441.320000   343
at_Risk              152.158756   611   3.073650   611  1188.878316   611
cant_loose           124.116883    77   9.116883    77  4099.450000    77
champions              7.119155   663  12.553544   663  6852.264167   663
hibernating          213.885714  1015   1.126108  1015   403.977836  1015
loyal_customers       36.287062   742   6.830189   742  2746.067353   742
need_attention        53.265700   207   2.449275   207  1060.357005   207
new_customers          8.580000    50   1.000000    50   386.199200    50
potential_loyalists   18.793037   517   2.017408   517   729.510986   517
promising             25.747126    87   1.000000    87   367.086782    87

In [24]:
rfm[rfm["segment"] == "champions"].index


Index([12360.0, 12395.0, 12415.0, 12429.0, 12431.0, 12433.0, 12471.0, 12472.0,
       12474.0, 12481.0,
       ...
       18144.0, 18168.0, 18170.0, 18219.0, 18225.0, 18226.0, 18229.0, 18245.0,
       18259.0, 18260.0],
      dtype='float64', name='Customer ID', length=663)

For example, let's say the marketing department requests information about the customers in the 'can't lose' segment. In this case, we need to extract the Customer ID information of the customers in this segment and send it to the marketing team.

In [25]:
rfm[rfm["segment"] == "cant_loose"].index

Index([12346.0, 12380.0, 12482.0, 12510.0, 12891.0, 12932.0, 13044.0, 13313.0,
       13680.0, 13782.0, 13799.0, 13856.0, 14025.0, 14063.0, 14160.0, 14221.0,
       14548.0, 14607.0, 14685.0, 14745.0, 15003.0, 15013.0, 15015.0, 15125.0,
       15141.0, 15222.0, 15306.0, 15321.0, 15359.0, 15369.0, 15372.0, 15443.0,
       15538.0, 15607.0, 15633.0, 15722.0, 15751.0, 15754.0, 15768.0, 15911.0,
       15912.0, 16027.0, 16032.0, 16158.0, 16177.0, 16197.0, 16335.0, 16467.0,
       16631.0, 16742.0, 16743.0, 16875.0, 16986.0, 17021.0, 17032.0, 17092.0,
       17113.0, 17157.0, 17188.0, 17230.0, 17268.0, 17426.0, 17448.0, 17454.0,
       17512.0, 17578.0, 17602.0, 17651.0, 17940.0, 17969.0, 17988.0, 18009.0,
       18051.0, 18064.0, 18094.0, 18251.0, 18258.0],
      dtype='float64', name='Customer ID')

In [26]:
new_df = pd.DataFrame()
new_df["cant_loose_customer_id"] = rfm[rfm["segment"] == "cant_loose"].index.astype("int")
new_df.to_csv("cant_loose_customer_id")
new_df.head()

,cant_loose_customer_id
0,12346
1,12380
2,12482
3,12510
4,12891


Alternatively, it is common to export the RFM dataframe that contains all segment information.

In [29]:
rfm.to_csv("rfm.csv", index=True, float_format="%.2f")

### 6. Functionizing the Entire Process

In [30]:
def create_rfm(dataframe, csv=False):

    # VERIYI HAZIRLAMA
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]

    # RFM METRIKLERININ HESAPLANMASI
    today_date = dt.datetime(2010, 12, 11)
    rfm = dataframe.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                                'Invoice': lambda num: num.nunique(),
                                                "TotalPrice": lambda price: price.sum()})
    rfm.columns = ['recency', 'frequency', "monetary"]
    rfm = rfm[(rfm['monetary'] > 0)]

    # RFM SKORLARININ HESAPLANMASI
    rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
    rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
    rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

    # cltv_df skorları kategorik değere dönüştürülüp df'e eklendi
    rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                        rfm['frequency_score'].astype(str))


    # SEGMENTLERIN ISIMLENDIRILMESI
    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }

    rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
    rfm = rfm[["recency", "frequency", "monetary", "segment"]]
    rfm.index = rfm.index.astype(int)

    if csv:
        rfm.to_csv("rfm.csv")

    return rfm